## proc directory
/proc: directory in charge of linux performance optimization

proc fidrectory has 3 parts
1. PID directory
2. information files /meminfo: prevents information from kernel
3. /sys: tunables

## sys
important interface for sys
- fs: filesystem
- kernel: for kernel
- net: networking
- vm: memory

swappiness: the williness of the kernel to swap
out memory pages, if memory pressure arises

### Systemctl service
```systemctl status systemd-sysct```
its configuration is in /etc/sysctl.conf<br>
changes to system is applied on boot<br>
you can change swappiness

Useful sysctl command

In [ ]:
%%bash
sysctl --help
sysctl -p /etc/sysctl.conf

In [ ]:
%%bash
# list sysctl prameters
sysctl -a

# Managing Kernel Module Parameters
- Use **modinfo** to find which parameters are available
- Use **modprobe module key=value** to specify a parameter in runtime
- Use /etc/modprobe.d/modulename.conf to specify a permanent parameter